# Timing of oab
## Run these once, at the start of the notebook

In [ ]:
import logging
import oab

In [1]:
import pickle
import time
from collections import Counter
from time import perf_counter

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from rich import print


def longrunning_function():
    for i in range(1, 11):
        time.sleep(i / i**2)

In [2]:
# make results dataframe
results_table = pd.DataFrame(columns=["Dataset", "Algo", "Class", "Time"])

## Personalize the following cell

In [3]:
# personalize this for running the notebook in different ways
dataset_name = "mnist"
algo_name = "RF"

my_class = 7

## Run these after each "personalization"
To get results on multiple datasets, algorithms, classes

### Cells *not* timed
Data loading and more

In [ ]:
(X_train, Y_train), (X_test, Y_test), (X_tree, Y_tree) = oab.get_data(dataset_name)
my_dom = oab.Domain(dataset_name, algo_name)
index = {}
points = {}
for my_class in range(10):
    index[my_class] = np.where(Y_test == my_class)[0]
my_dom.load()

### Timed cells

In [ ]:
start = perf_counter()

longrunning_function()  # here the actual explanation of a certain number of records

end = perf_counter()
# compute time
execution_time = end - start
# append into results_table
results_table = results_table.append(
    {
        "Dataset": dataset_name,
        "Algo": algo_name,
        "Class": my_class,
        "Time": execution_time,
    },
    ignore_index=True,
)

## Run this to show the results table

In [7]:
results_table

,dataset,algo,class,time
